# Nueral Network
This is another model we are testing for our application. A nueral network may perform better than an XG Boost model, so we will load the same data and see if we can get better results.

In [2]:
# required imports
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import train_test_split

# tf imports
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.optimizers import RectifiedAdam, Lookahead
from sklearn.base import BaseEstimator
from tensorflow_addons.activations import gelu


ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
def create_model(grid_params, in_dim, out_dim, patience=20, loss='rmse', activation='sigmoid'):
    
    mul_input = grid_params['mul_input']
    n_layer = grid_params['n_layer']
    
    first_layer_size = int(in_dim*mul_input)
    hidden_layers = []
    for i_layer in range(n_layer, 0, -1):
        layer_size = int(((first_layer_size - out_dim) / n_layer) * i_layer + out_dim)
        hidden_layers.append(layer_size)

    print("Input dim:" + str(in_dim))
    print("Hidden Layers:" + str(hidden_layers))
    print("Output dim:" + str(out_dim))

    model = Sequential()
    
    model.add(Dense(in_dim,input_shape=[in_dim],activation=gelu))
    #model.add(BatchNormalization())
    model.add(Dropout(.5))
    
    for layer in hidden_layers:
        model.add(Dense(layer,activation=gelu))
        #model.add(BatchNormalization())
        model.add(Dropout(.5))
    
    model.add(Dense(out_dim, activation=activation))
    
    radam = RectifiedAdam()
    ranger = Lookahead(radam, sync_period=6, slow_step_size=0.5)
    optimizer = ranger#Adam(learning_rate=0.001)
    
    es = EarlyStopping(monitor='val_loss', verbose=1, mode='min', patience=patience, restore_best_weights=True)
    es.set_model(model)

    model.compile(optimizer=optimizer, loss=[loss], metrics=[])
    
    return model, [ es ]